In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))

In [ ]:
# from prompts.promptmanager import PromptManager
# from prompts.prompt_type.fairy.FairyPromptType import FairyPromptType
# from agents.fairy.util import get_groq_llm_lc
# from langchain_core.messages import HumanMessage, SystemMessage
# from agents.fairy.fairy_state import FairyDungeonIntentOutput
# from enums.LLM import LLM

# expect_multi_question = "HP 포션좀 먹고 싶어, 물약좀 먹으면서 저 슬라임 공략법에 대해 알려줘"
# expect_monster = "저거 공략법좀 알려줘"
# expect_event = "이 석상을 부시면 어떻게 되는거야?"
# expect_navigator= "현재 방에서 어디로 가야돼?"
# expect_interaction_handler = "방 불좀 켜줘"
# expect_small_talk = "심심행"
# expect_unknown_intent = "al;sdas 내가 낀 아이템을 사용해"
# expect_game_system = "게임 어떻게 꺼?"

# expect_super_multi = "내가 낀 아이템을 사용해 그리고 저 몬스터 공략법좀 알려줘 그리고 이 석상을 어떻게 부셔? 다음 던전 방에는 뭐고? 그리고 심심해 그리고 게임 어떻게 종료해?"

# system_prompt = PromptManager(FairyPromptType.FAIRY_DUNGEON_INTENT).get_prompt()
# messages = [
#     SystemMessage(content=system_prompt),
#     HumanMessage(content="뭙야")
# ] 

# parser_llm = get_groq_llm_lc(model=LLM.LLAMA_3_1_8B_INSTANT, max_token=43).with_structured_output(FairyDungeonIntentOutput)
# result = parser_llm.invoke(messages)
# print(result)

In [1]:
%%writefile fairy_dungeon_agent.py
from agents.fairy.fairy_state import (
    FairyDungeonIntentOutput,
    FairyDungeonState,
    FairyDungeonIntentType,
)
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.types import interrupt
from agents.fairy.cache_data import reverse_questions, GAME_SYSTEM_INFO
from prompts.promptmanager import PromptManager
from prompts.prompt_type.fairy.FairyPromptType import FairyPromptType
import random, asyncio, time
from agents.fairy.util import (
    add_ai_message,
    add_human_message,
    get_groq_llm_lc,
    find_monsters_info,
)
from core.common import get_inventory_items
from enums.LLM import LLM
from langchain.chat_models import init_chat_model
from typing import List
from db.RDBRepository import RDBRepository
from db.rdb_entity.DungeonRow import DungeonRow
from agents.fairy.dynamic_prompt import (
    dungeon_spec_prompt,
    item_spec_prompt,
    monster_spec_prompt,
)
from agents.fairy.util import (
    contains_hanja,
    replace_hanja_naively,
    get_human_few_shot_prompts,
    describe_dungeon_row,
)
from agents.fairy.dungeon.fairy_dungeon_model_logics import FairyDungeonIntentModel
import asyncio

intent_llm = get_groq_llm_lc(model=LLM.LLAMA_3_1_8B_INSTANT, max_token=43)
# action_llm = get_groq_llm_lc(max_token=80, temperature=0)
action_llm = init_chat_model(
    model=LLM.GROK_4_FAST_NON_REASONING, max_tokens=80, temperature=0
)
small_talk_llm = init_chat_model(model=LLM.GROK_4_FAST_NON_REASONING, max_tokens=120)
rdb_repository = RDBRepository()
intent_model = FairyDungeonIntentModel()


def _rdb_fairy_messages_bg(user_args, ai_args):
    try:
        rdb_repository.insert_fairy_message(**user_args)
        rdb_repository.insert_fairy_message(**ai_args)
    except Exception as e:
        print("fairy_message insert failed:", e)


async def get_monsters_info(target_monster_ids: List[int]):
    monster_prompt = monster_spec_prompt.format(
        monster_infos_json=find_monsters_info(target_monster_ids)
    )
    return monster_prompt


async def get_event_info(dungeon_row: DungeonRow, curr_room_id: int):
    event = dungeon_row.event
    if event is None:
        return '이벤트가 생성되지 않았습니다. 페이몬은 "응? 확인된 이벤트는 아직 없어!"라고 장난스럽게 답해주세요.'

    if event.room_id != curr_room_id:
        return '이벤트방에 입장하지 않아서 정보를 확인할 수 없습니다. 페이몬은 "아직은 아무일 없어보여! 무슨 사건이 일어날 때 말해!" 라는식으로 장난스럽게 답해주세요.'

    print("이벤트", event)
    return event


async def dungeon_navigator(dungeon_row: DungeonRow, curr_room_id: int):
    summary_info = dungeon_row.summary_info
    dungeon_prompt = describe_dungeon_row(
        curr_room_id, dungeon_row.balanced_map, dungeon_row.floor
    )
    dungeon_map_prompt = (
        f"        <DUNGEON_MAP>\n" f"{dungeon_prompt}\n" f"        </DUNGEON_MAP>"
    )

    dungeon_summary_prompt = (
        f"        <DUNGEON_SUMMARY>\n" f"{summary_info}\n" f"        </DUNGEON_SUMMARY>"
    )

    dungeon_current_prompt = (
        f"        <CURRENT_ROOM_ID>\n" f"{curr_room_id}\n" f"        </CURRENT_ROOM_ID>"
    )
    total_prompt = (
        dungeon_map_prompt
        + "\n"
        + dungeon_summary_prompt
        + "\n"
        + dungeon_current_prompt
    )
    return total_prompt


async def create_interaction(inventory_ids):
    item_prompt = item_spec_prompt.format(items_json=get_inventory_items(inventory_ids))
    inventory_prompt = (
        f"        <INVENTORY_ITEMS>\n" f"{item_prompt}\n" f"        </INVENTORY_ITEMS>"
    )
    result = inventory_prompt
    return result


async def get_system_info():
    return GAME_SYSTEM_INFO


async def _clarify_intent(query) -> FairyDungeonIntentOutput:
    intent_prompt = PromptManager(FairyPromptType.FAIRY_DUNGEON_INTENT).get_prompt()
    messages = [SystemMessage(content=intent_prompt), HumanMessage(content=query)]
    parser_llm = intent_llm.with_structured_output(FairyDungeonIntentOutput)
    intent_output: FairyDungeonIntentOutput = await parser_llm.ainvoke(messages)
    # raw_labels, _ = intent_model.predict(query)
    # enum_list = FairyDungeonIntentModel.parse_intents_to_enum(raw_labels)
    # intent_output: FairyDungeonIntentOutput = FairyDungeonIntentOutput(intents=enum_list)
    # print("전체 의도::", intent_output)
    return intent_output


async def analyze_intent(state: FairyDungeonState):
    start = time.perf_counter()
    last = state["messages"][-1]
    last_message = last.content
    clarify_intent_type: FairyDungeonIntentOutput = await _clarify_intent(last_message)

    if clarify_intent_type.intents[0] == FairyDungeonIntentType.UNKNOWN_INTENT:
        clarification = reverse_questions[random.randint(0, 148)]
        user_resp = interrupt(clarification)
        return {
            "messages": [
                add_ai_message(
                    content=clarification, intent_types=clarify_intent_type.intents
                ),
                add_human_message(content=user_resp),
            ],
            "intent_types": clarify_intent_type.intents,
            "is_multi_small_talk": False,
        }
    latency = time.perf_counter() - start
    return {
        "intent_types": clarify_intent_type.intents,
        "latency_analyze_intent": latency,
    }


def check_condition(state: FairyDungeonState):
    intent_types = state.get("intent_types", [])
    if intent_types[0] == FairyDungeonIntentType.UNKNOWN_INTENT:
        return "retry"
    return "continue"


async def fairy_action(state: FairyDungeonState):
    start = time.perf_counter()
    intent_types = state.get("intent_types")
    dungenon_player = state["dungenon_player"]
    target_monster_ids = state.get("target_monster_ids", [])
    currRoomId = dungenon_player.currRoomId
    dungeon_row = rdb_repository.get_current_dungeon_by_player(
        dungenon_player.playerId, dungenon_player.heroineId
    )
    messages = state["messages"]
    INTENT_HANDLERS = {
        FairyDungeonIntentType.MONSTER_GUIDE: lambda: get_monsters_info(
            target_monster_ids
        ),
        FairyDungeonIntentType.EVENT_GUIDE: lambda: get_event_info(
            dungeon_row, currRoomId
        ),
        FairyDungeonIntentType.DUNGEON_NAVIGATOR: lambda: dungeon_navigator(
            dungeon_row, currRoomId
        ),
        FairyDungeonIntentType.INTERACTION_HANDLER: lambda: create_interaction(
            dungenon_player.inventory
        ),
        FairyDungeonIntentType.USAGE_GUIDE: get_system_info,
    }
    INTENT_LABELS = {
        FairyDungeonIntentType.MONSTER_GUIDE: "MONSTER_GUIDE",
        FairyDungeonIntentType.EVENT_GUIDE: "EVENT_GUIDE",
        FairyDungeonIntentType.DUNGEON_NAVIGATOR: "DUNGEON_NAVIGATOR",
        FairyDungeonIntentType.INTERACTION_HANDLER: "INTERACTION_HANDLER",
        FairyDungeonIntentType.USAGE_GUIDE: "USAGE_GUIDE",
    }

    handlers = [INTENT_HANDLERS[i]() for i in intent_types if i in INTENT_HANDLERS]
    results = await asyncio.gather(*handlers)

    prompt_info = ""
    idx = 0
    for i, index in enumerate(intent_types):
        handler = INTENT_HANDLERS.get(index)
        if not handler:
            continue

        value = results[idx]
        label = INTENT_LABELS.get(index, "정보")
        if i == 0:
            prompt_info += f"    <{label}>\n{value}\n    </{label}>"
        else:
            prompt_info += f"\n    <{label}>\n{value}\n    </{label}>"
        idx += 1

    pretty_dungenon_player = dungenon_player.model_dump_json(indent=2)
    system_prompt = PromptManager(FairyPromptType.FAIRY_DUNGEON_SYSTEM).get_prompt()

    question = messages[-1].content

    fewshots = get_human_few_shot_prompts(intent_types)

    human_prompt = PromptManager(FairyPromptType.FAIRY_DUNGEON_HUMAN).get_prompt(
        dungenon_player=pretty_dungenon_player,
        use_intents=[rt.value if hasattr(rt, "value") else rt for rt in intent_types],
        ability_examples=fewshots,
        info=prompt_info,
        question=question,
    )

    print("총 질문", human_prompt)

    if intent_types[0] == FairyDungeonIntentType.SMALLTALK and len(intent_types) == 1:
        ai_answer = small_talk_llm.invoke(
            [SystemMessage(content=system_prompt)]
            + messages
            + [HumanMessage(content=human_prompt)]
        )
    else:
        ai_answer = action_llm.invoke(
            [
                SystemMessage(content=system_prompt),
                HumanMessage(content=human_prompt),
            ]
        )
        if contains_hanja(ai_answer.content):
            ai_answer.content = replace_hanja_naively(ai_answer.content)

    asyncio.create_task(
        asyncio.to_thread(
            _rdb_fairy_messages_bg,
            {
                "sender_type": "USER",
                "message": question,
                "context_type": "DUNGEON",
                "player_id": dungenon_player.playerId,
                "heroine_id": dungenon_player.heroineId,
                "intent_type": None,
            },
            {
                "sender_type": "AI",
                "message": ai_answer.content,
                "context_type": "DUNGEON",
                "player_id": dungenon_player.playerId,
                "heroine_id": dungenon_player.heroineId,
                "intent_type": intent_types,
            },
        )
    )
    latency = time.perf_counter() - start
    return {
        "messages": [
            add_ai_message(content=ai_answer.content, intent_types=intent_types)
        ],
        "latency_fairy_action": latency,
    }


from langgraph.graph import START, END, StateGraph

graph_builder = StateGraph(FairyDungeonState)

graph_builder.add_node("analyze_intent", analyze_intent)
graph_builder.add_node("fairy_action", fairy_action)
graph_builder.add_edge(START, "analyze_intent")

graph_builder.add_conditional_edges(
    "analyze_intent",
    check_condition,
    {
        "retry": "analyze_intent",
        "continue": "fairy_action",
    },
)
graph_builder.add_edge("fairy_action", END)
graph_builder.compile()

Overwriting fairy_dungeon_agent.py


In [2]:
import os, sys
from pathlib import Path

def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")

src_path = find_src_folder()
sys.path.append(str(src_path))
from agents.fairy.dungeon.fairy_dungeon_agent import graph_builder as dungeon_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from agents.fairy.fairy_state import DungeonPlayerState
import random

player = DungeonPlayerState(
    playerId="101",
    heroineId=1,
    currRoomId=1,
    difficulty=0,
    hp = 250,
    moveSpeed = 1,
    attackSpeed = 1.0,
    # weapon=MockFactory.create_weapon(),
    inventory=[0],
)
memory = MemorySaver()
graph = dungeon_graph_builder.compile(memory)

config = {"configurable": {"thread_id": "user_1"}}
inputs = {
    "messages": [add_human_message(content="장착중인 아이템이 뭐야?")],    
    "dungenon_player": player,
    "target_monster_ids":[],
}
await graph.ainvoke(inputs, config=config)

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


총 질문 <Used Abilities>
['INTERACTION_HANDLER']
</Used Abilities>

<Heroine Info>
{
  "playerId": "101",
  "heroineId": 1,
  "currRoomId": 1,
  "difficulty": 0,
  "hp": 250,
  "moveSpeed": 1.0,
  "attackSpeed": 1.0,
  "weapon": null,
  "inventory": [
    0
  ]
}
</Heroine Info>

<Current Situation>
    <INTERACTION_HANDLER>
        <INVENTORY_ITEMS>

Below is the data specification describing Items and Weapons
that the player can own, obtain, and equip.

All items share common fields (name / ID / type / rarity).
If the item is a weapon-type item, detailed weapon stats are included
in the weapon field.

■ items (ItemData[])
  A list of items.
  (e.g., inventory contents, drop rewards, shop listings)

  ● itemId (int)
      Unique item ID

  ● itemName (string | null)
      Item name (may be null)

  ● itemType (int)
      Item type
        0: Accessory
        1: One-handed sword
        2: Dual blades
        3: Greatsword
        4: Blunt weapon (two-handed)

  ● rarity (int, 0–3)
     

{'messages': [HumanMessage(content='장착중인 아이템이 뭐야?', additional_kwargs={'created_at': '2025-12-18T19:18:45.471496'}, response_metadata={}, id='089023af-1d8e-4b73-aebb-3682e15e0b3b'),
  AIMessage(content='지금은 무기가 장착되어 있지 않아! 인벤토리에 일반 한손검이 있으니 그걸 장착해 볼래?', additional_kwargs={'created_at': '2025-12-18T19:18:47.025603', 'intent_types': ['INTERACTION_HANDLER']}, response_metadata={}, id='00bddab5-5bdd-404f-acf0-948b7177e20b')],
 'intent_types': [<FairyDungeonIntentType.INTERACTION_HANDLER: 'INTERACTION_HANDLER'>],
 'dungenon_player': DungeonPlayerState(playerId='101', heroineId=1, currRoomId=1, difficulty=0, hp=250, moveSpeed=1.0, attackSpeed=1.0, weapon=None, inventory=[0]),
 'target_monster_ids': [],
 'latency_analyze_intent': 0.44999929199548205,
 'latency_fairy_action': 1.1019304170040414}